In [1]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X_train=mnist.train.images
y_train=mnist.train.labels
X_test=mnist.test.images
y_test=mnist.test.labels

In [3]:
import tensorflow as tf
from IPython.display import clear_output
#MNIST
n_inputs=28*28 
n_hidden1=1000
n_hidden2=500
n_outputs=10

n_epochs=60
batch_size=500

X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

#or use the handy tf dense function
tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

#apply l1 regularizer and he init method
from functools import partial
he_init = tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG") # he init method
scale=0.001
my_dense=partial(tf.layers.dense,activation=tf.nn.elu,
                 kernel_regularizer=tf.contrib.layers.l1_regularizer(scale),
                 kernel_initializer=he_init)

with tf.name_scope("dnn"):
    dropout_rate=0.5
    training=tf.placeholder_with_default(False,shape=(),name="training")
    with tf.device("/gpu:0"):
        hidden1=my_dense(X,n_hidden1,name="hidden1",kernel_initializer=he_init,activation=tf.nn.elu) #use ELU
        #dropout1
        hidden1_drop=tf.layers.dropout(hidden1,dropout_rate,training=training)
    with tf.device("/gpu:1"):
        hidden2=my_dense(hidden1_drop,n_hidden2,name="hidden2",kernel_initializer=he_init,activation=tf.nn.elu) #ELU
        #dropout
        hidden2_drop=tf.layers.dropout(hidden2,dropout_rate,training=training)
    with tf.device("/cpu:0"):
        logits=my_dense(hidden2_drop,n_outputs, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale),
                 kernel_initializer=he_init,name="outputs")
    
with tf.name_scope("loss"):
    with tf.device("/cpu:0"):
        xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
        loss=tf.reduce_mean(xentropy,name="loss")
    
learning_rate=0.001
with tf.name_scope("train"):
    #use Adam
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)

with tf.name_scope("eval"):
    with tf.device("/cpu:0"):
        correct=tf.nn.in_top_k(logits,y,1)
        accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
init=tf.global_variables_initializer()
saver=tf.train.Saver()


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch,training:True}) # dropout=True
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        clear_output()
        print(epoch,"Train accuracy: ",acc_train," Test accuracy: ",acc_test,end="\r")
    save_path=saver.save(sess,"./model_checkpoints/mnist_final.ckpt")

InternalError: Blas GEMM launch failed : a.shape=(500, 784), b.shape=(784, 1000), m=500, n=1000, k=784
	 [[Node: dnn/hidden1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_X_0_0/_21, hidden1/kernel/read)]]
	 [[Node: train/gradients/dnn/hidden2/BiasAdd_grad/BiasAddGrad/_91 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:1", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_210_train/gradients/dnn/hidden2/BiasAdd_grad/BiasAddGrad", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:1"]()]]

Caused by op 'dnn/hidden1/MatMul', defined at:
  File "d:\programdata\anaconda3\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\programdata\anaconda3\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\developing\deep-learning\tf-gpu\lib\site-pac